In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import logging
import pickle
import numpy as np

class CustomModel(nn.Module):
    def __init__(self, base_model, num_labels): 
        super(CustomModel, self).__init__()  
        self.base_model = base_model 
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(base_model.config.hidden_size, num_labels)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}
# Load data
train_path = "/kaggle/input/maqa-dataset/Unbalanced/Unbalanced/MAQA_Train.xlsx"
test_path = "/kaggle/input/maqa-dataset/Unbalanced/Unbalanced/MAQA_Test.xlsx"
train_df = pd.read_excel(train_path)
test_df = pd.read_excel(test_path)
all_data = pd.concat([train_df, test_df], ignore_index=True)
all_data = all_data[['q_body', 'category']]

valid_categories = [
    "امراض نسائية",
    "امراض العضلات والعظام و المفاصل",
    "امراض الجهاز الهضمي",
    "الامراض الجنسية",
    "طب الاسنان",
    "امراض القلب و الشرايين",
    "امراض العيون",
    "انف اذن وحنجرة",
    "جراحة تجميل",
    "امراض الدم"
]

all_data = all_data[all_data["category"].isin(valid_categories)]
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)
train_df, test_df = train_test_split(all_data, test_size=0.2, random_state=42, stratify=all_data['category'])
train_df["text"] = train_df["q_body"]
test_df["text"] = test_df["q_body"]

category_mapping = {cat: i for i, cat in enumerate(valid_categories)}
train_df['label'] = train_df['category'].map(category_mapping)
test_df['label'] = test_df['category'].map(category_mapping)

model_name = "monologg/biobert_v1.1_pubmed"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_dict({"text": train_df["text"].tolist(), "label": train_df["label"].tolist()})
test_dataset = Dataset.from_dict({"text": test_df["text"].tolist(), "label": test_df["label"].tolist()})

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


🔹 Training Dataset Class Distribution:
category
امراض نسائية                       56128
انف اذن وحنجرة                     15646
امراض العضلات والعظام و المفاصل    14848
امراض العيون                       14639
امراض القلب و الشرايين             12756
امراض الجهاز الهضمي                12708
الامراض الجنسية                     8874
طب الاسنان                          8805
جراحة تجميل                         7875
امراض الدم                          6861
Name: count, dtype: int64

🔹 Test Dataset Class Distribution:
category
امراض نسائية                       14032
انف اذن وحنجرة                      3912
امراض العضلات والعظام و المفاصل     3712
امراض العيون                        3660
امراض القلب و الشرايين              3190
امراض الجهاز الهضمي                 3177
الامراض الجنسية                     2219
طب الاسنان                          2202
جراحة تجميل                         1969
امراض الدم                          1716
Name: count, dtype: int64


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/159140 [00:00<?, ? examples/s]

Map:   0%|          | 0/39789 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-2-48adf45f4208>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("\n🔹 Starting Training...")
trainer.train()

test_metrics = trainer.evaluate(test_dataset)
print("\n🔹 Test Metrics:", test_metrics)

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

print("\n🔹 Confusion Matrix:")
print(confusion_matrix(labels, preds))

print("\n🔹 Classification Report:")
print(classification_report(labels, preds))


🔹 Starting Training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.507600,0.522392,0.840835,0.841410,0.840835,0.840106
2,0.366400,0.378196,0.881048,0.879337,0.881048,0.879296
3,0.336100,0.348277,0.891427,0.891271,0.891427,0.890671
4,0.312700,0.325073,0.898791,0.899630,0.898791,0.898556
5,0.236200,0.315732,0.902636,0.902260,0.902636,0.901884
6,0.197900,0.314258,0.904421,0.903708,0.904421,0.903655
7,0.184600,0.311632,0.906255,0.905171,0.906255,0.905081
8,0.151100,0.320682,0.905703,0.904696,0.905703,0.904561
9,0.156400,0.326859,0.906834,0.905602,0.906834,0.905728
10,0.145800,0.330647,0.906834,0.905774,0.906834,0.905894



🔹 Test Metrics: {'eval_loss': 0.33064693212509155, 'eval_accuracy': 0.9068335469602151, 'eval_precision': 0.9057742609146948, 'eval_recall': 0.9068335469602151, 'eval_f1': 0.9058936545235001, 'eval_runtime': 147.1435, 'eval_samples_per_second': 270.41, 'eval_steps_per_second': 2.82, 'epoch': 10.0}

🔹 Confusion Matrix:
[[13514    95    83   195    34    34    12    32    27     6]
 [   78  3456    20    27    10    53     5    26    14    23]
 [  107    40  2763    38    32    84    11    55    23    24]
 [  451    39    30  1639    12     7     1     4    22    14]
 [   53    12    18     8  2067     1     5    27    11     0]
 [   44   106    74    10     5  2678    11    44    18   200]
 [    8    11    16     3     1    15  3467    33    73    33]
 [   26    29    48    10    45    37    30  3613    54    20]
 [  173    57    22    20    29    14    47    64  1515    28]
 [   21    20    35    13     5   170    21    34    27  1370]]

🔹 Classification Report:
              precisio

In [4]:
# 🔹 Save Model & Tokenizer
def save_complete_model(model, tokenizer, category_mapping, save_path):
    model.base_model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    
    classifier_state = {
        'classifier_state': model.classifier.state_dict(),
        'num_labels': model.classifier.out_features
    }
    torch.save(classifier_state, f"{save_path}/classifier_state.pt")
    
    with open(f"{save_path}/category_mapping.pkl", "wb") as f:
        pickle.dump(category_mapping, f)

# Save the model
save_complete_model(trainer.model, tokenizer, category_mapping, "BioBert")

# 🔹 Load the Model for Inference
def load_complete_model(model_path):
    base_model = AutoModel.from_pretrained(model_path)
    classifier_state = torch.load(f"{model_path}/classifier_state.pt", map_location=torch.device('cpu'))
    model = CustomModel(base_model, classifier_state['num_labels'])
    model.classifier.load_state_dict(classifier_state['classifier_state'])
    model.eval()
    return model

In [5]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("BioBert")

# Load the trained model
model2 = load_complete_model("BioBert")
print("✅ Model Loaded Successfully!")

# 🔹 Function for Predictions
def predict_category(text, model, tokenizer, category_mapping):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    logits = outputs["logits"]
    predicted_label = torch.argmax(logits, dim=-1).item()
    category_mapping_reverse = {v: k for k, v in category_mapping.items()}
    return category_mapping_reverse[predicted_label]



✅ Model Loaded Successfully!


<ipython-input-4-e42658041941>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier_state = torch.load(f"{model_path}/classifier_state.pt", map_location=torch.device

In [6]:
# 🔹 Sample Predictions
test_samples = ["انيميا حاده", "التهاب في المعده", "جفاف العين", "نزله برد", "كسر في المفصل"]
for text in test_samples:
    predicted_category = predict_category(text, model2, tokenizer, category_mapping)
    print(f"\n🔹 Input: {text}")
    print(f"Predicted Category: {predicted_category}")

# 🔹 Zip Model for Download
shutil.make_archive("BioBert", 'zip', "BioBert")
print("\n✅ Model Saved & Zipped for Download!")

# Provide a download link
from IPython.display import FileLink
FileLink(r'BioBert.zip')


🔹 Input: انيميا حاده
Predicted Category: امراض الدم

🔹 Input: التهاب في المعده
Predicted Category: امراض الجهاز الهضمي

🔹 Input: جفاف العين
Predicted Category: امراض العيون

🔹 Input: نزله برد
Predicted Category: انف اذن وحنجرة

🔹 Input: كسر في المفصل
Predicted Category: امراض العضلات والعظام و المفاصل

✅ Model Saved & Zipped for Download!


/kaggle/working/BioBert.zip